In [5]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

# Step 1: Dataset Splitting
def split_dataset(source_dir, train_dir, validation_dir, test_size=0.2):
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    if not os.path.exists(validation_dir):
        os.makedirs(validation_dir)

    for class_name in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_name)
        if os.path.isdir(class_path):
            images = [img for img in os.listdir(class_path) 
                      if os.path.isfile(os.path.join(class_path, img)) and img.lower().endswith(('.png', '.jpg', '.jpeg'))]
            
            if len(images) == 0:  # Handle empty directories
                print(f"Warning: No valid images found in {class_path}. Skipping...")
                continue

            train, val = train_test_split(images, test_size=test_size, random_state=42)

            class_train_dir = os.path.join(train_dir, class_name)
            class_val_dir = os.path.join(validation_dir, class_name)
            os.makedirs(class_train_dir, exist_ok=True)
            os.makedirs(class_val_dir, exist_ok=True)

            for img in train:
                shutil.copy(os.path.join(class_path, img), os.path.join(class_train_dir, img))
            for img in val:
                shutil.copy(os.path.join(class_path, img), os.path.join(class_val_dir, img))

    print("Dataset split completed.")

source_dir = r"D:\Final Year Project\Mango Dataset"
train_dir = r"D:\Final Year Project\Mango Dataset\train"
validation_dir = r"D:\Final Year Project\Mango Dataset\validation"
split_dataset(source_dir, train_dir, validation_dir)

# Step 2: Data Preparation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Step 3: Model Creation
model = models.Sequential([
    layers.Input(shape=(150, 150, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 4: Model Training
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# Save the model
model.save("mango_disease_detection_model.keras")

# Step 5: Testing the Model
from tensorflow.keras.utils import load_img, img_to_array

def predict_image(image_path, model, class_indices):
    img = load_img(image_patha, target_size=(150, 150))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])
    class_labels = {v: k for k, v in class_indices.items()}
    return class_labels[predicted_class]

image_path = r"D:\Final Year Project\mango.jpg"
predicted_disease = predict_image(image_path, model, train_generator.class_indices)
print(f"Predicted Disease: {predicted_disease}")


Dataset split completed.
Found 7200 images belonging to 8 classes.
Found 1800 images belonging to 8 classes.
Epoch 1/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 338s 1s/step - accuracy: 0.3435 - loss: 1.5947 - val_accuracy: 0.5494 - val_loss: 1.3452
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 259s 1s/step - accuracy: 0.6627 - loss: 0.8913 - val_accuracy: 0.6722 - val_loss: 1.0032
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 255s 1s/step - accuracy: 0.7663 - loss: 0.6284 - val_accuracy: 0.6811 - val_loss: 1.2763
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 262s 1s/step - accuracy: 0.7966 - loss: 0.5464 - val_accuracy: 0.7561 - val_loss: 0.6654
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 291s 1s/step - accuracy: 0.8541 - loss: 0.4158 - val_accuracy: 0.8578 - val_loss: 0.4497
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 218s 967ms/step - accuracy: 0.8862 - loss: 0.3360 - val_accuracy: 0.8944 - val_loss: 0.3007
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 206s 917ms/step - accuracy: 0.8996 - loss: 0.2798 - val_accuracy: 0.8661 -

In [7]:
image_path = r"D:\Final Year Project\12.jpg"
predicted_disease = predict_image(image_path, model, train_generator.class_indices)
print(f"Predicted Disease: {predicted_disease}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Predicted Disease: Bacterial Canker
